
# Sinasodal Model:  Analysis and Synthesis

# <h1>Table of Contents<span class="tocSkip"></span></h1>
# <div class="toc"><ul class="toc-item"><li><span><a href="#Example-with-audio-signal" data-toc-modified-id="Example-with-audio-signal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Example with audio signal</a></span></li><li><span><a href="#Analysis-of-Audio-with-only-20-sinasodals" data-toc-modified-id="Analysis-of-Audio-with-only-20-sinasodals-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Analysis of Audio with only 20 sinasodals</a></span></li><li><span><a href="#Synthesis-of-Audio-from-20-sinasodal-tracks" data-toc-modified-id="Synthesis-of-Audio-from-20-sinasodal-tracks-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Synthesis of Audio from 20 sinasodal tracks</a></span></li><li><span><a href="#Residual:-Discarded-part-of-Audio---mostly-hissing-sound" data-toc-modified-id="Residual:-Discarded-part-of-Audio---mostly-hissing-sound-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Residual: Discarded part of Audio - mostly hissing sound</a></span></li></ul></div>

# # Sinasodal Model:  Analysis and Synthesis


## Model
embed_audio('singing_female_recons', attribute = c("controls", "loop"))



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import requests, logging
from scipy.io import wavfile
#import IPython


import spkit as sp
sp.__version__



# ## Example with audio signal

path1 = 'https://github.com/Nikeshbajaj/web-data/blob/main/sounds/violin-B3.wav?raw=true'
path2 = 'https://github.com/Nikeshbajaj/web-data/blob/main/sounds/singing-female.wav?raw=true'
print(path2)

# embed_audio(path2, attribute = c("controls", "loop"))
# ![]('https://github.com/Nikeshbajaj/web-data/blob/main/sounds/singing-female.wav?raw=true')


logging.info(f"Downloading wav file from: {path2}")
req = requests.get(path2)
with open('myfile.wav', 'wb') as f:
        f.write(req.content)



fs, x = wavfile.read('myfile.wav')
t = np.arange(len(x))/fs

print(x.shape, fs)



mXt,pXt = sp.stft_analysis(x, winlen=441, overlap=220,window='blackmanharris',nfft=None)


plt.figure(figsize=(13,6))
plt.subplot(211)
plt.plot(t,x)
plt.xlim([t[0],t[-1]])
plt.grid()
plt.title('Original audio')
#plt.xlabel('time (s)')
plt.ylabel('amplitude (μV)')

plt.subplot(212)
plt.imshow(mXt.T,aspect='auto',origin='lower',cmap='jet',extent=[t[0],t[-1],0,fs/2])
plt.title('Spectrogram')
#plt.xlabel('time (s)')
plt.ylabel('frequency (Hz)')

plt.tight_layout()
plt.show()

# IPython.display.Audio(x,rate=fs)


# Original Audio
# 
# <audio controls="controls">
#       <source src="https://github.com/Nikeshbajaj/web-data/blob/main/sounds/singing-female.wav?raw=true" type="audio/wav"> 
# </audio>
# 
# https://github.com/Nikeshbajaj/web-data/blob/main/sounds/singing-female.wav?raw=true

## Analysis of Audio with only 20 sinasodals



In [ ]:
N = 20


fXst, mXst, pXst = sp.sineModel_analysis(x,fs,winlen=3001,overlap=750,
                            window='blackmanharris', nfft=None, thr=-10, 
                            maxn_sines=N,minDur=0.01, freq_devOffset=10,freq_devSlope=0.1)

print(fXst.shape, mXst.shape, pXst.shape)


# plotting only frequency which have magnitude > 0dB


fXt1 = (fXst.copy())*(mXst>0)
fXt1[fXt1==0]=np.nan



plt.figure(figsize=(13,9))
plt.subplot(311)
plt.plot(t,x)
plt.xlim([t[0],t[-1]])
plt.grid()
plt.title('Original Audio')
#plt.xlabel('time (s)')
plt.ylabel('amplitude (μV)')


plt.subplot(312)
plt.imshow(mXt.T,aspect='auto',origin='lower',cmap='jet',extent=[t[0],t[-1],0,fs/2])
plt.title('Spectrogram')
#plt.xlabel('time (s)')
plt.ylabel('frequency (Hz)')


plt.subplot(313)
tx = t[-1]*np.arange(fXt1.shape[0])/fXt1.shape[0]

plt.plot(tx,fXt1,'-k',alpha=0.5)
#plt.ylim([0,fs/2])
plt.xlim([0,tx[-1]])

plt.title(f'Sinasodals tracks: n={N}')
plt.xlabel('time (s)')
plt.ylabel('frequency (Hz)')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


# ## Synthesis of Audio from 20 sinasodal tracks

Xr = sp.sineModel_synthesis(fXst, mXst, pXst,fs,overlap=750)
print(Xr.shape)



mXrt,pXrt = sp.stft_analysis(Xr, winlen=441, overlap=220,window='blackmanharris',nfft=None)


plt.figure(figsize=(13,6))
plt.subplot(211)
plt.plot(t,Xr[:len(x)])
plt.xlim([t[0],t[-1]])
plt.grid()
plt.title(f'Reconstructed Audio from {N} Sinasodals: $x_r(t)$')
#plt.xlabel('time (s)')
plt.ylabel('amplitude (μV)')

plt.subplot(212)
plt.imshow(mXrt.T,aspect='auto',origin='lower',cmap='jet',extent=[t[0],t[-1],0,fs/2])
plt.title(r'Spectrogram of $x_r(t)$')
#plt.xlabel('time (s)')
plt.ylabel('frequency (Hz)')

plt.tight_layout()
plt.show()

#IPython.display.Audio(Xr,rate=fs)


# Reconstructed Audio
# 
# <audio controls="controls">
#       <source src="https://raw.githubusercontent.com/Nikeshbajaj/spkit/master/spkit/data/singing_female_recons.wav" type="audio/wav">
# </audio>
# 
# https://raw.githubusercontent.com/Nikeshbajaj/spkit/master/spkit/data/singing_female_recons.wav

# ## Residual: Discarded part of Audio - mostly hissing sound


Xd = x - Xr[:len(x)]
Xd.shape



mXdt,pXdt = sp.stft_analysis(Xd, winlen=441, overlap=220,window='blackmanharris',nfft=None)


plt.figure(figsize=(13,6))
plt.subplot(211)
plt.plot(t,Xd)
plt.xlim([t[0],t[-1]])
plt.grid()
plt.title(r'Residual: Discarded part of Audio: $x_d(t) = x(t)-x_r(t)$')
#plt.xlabel('time (s)')
plt.ylabel('amplitude (μV)')

plt.subplot(212)
plt.imshow(mXdt.T,aspect='auto',origin='lower',cmap='jet',extent=[t[0],t[-1],0,fs/2])
plt.title(r'Spectrogram of $x_d(t)$')
#plt.xlabel('time (s)')
plt.ylabel('frequency (Hz)')

plt.tight_layout()
plt.show()

#IPython.display.Audio(Xd,rate=fs)


# Residual Audio   
# <audio controls="controls">
#       <source src="https://raw.githubusercontent.com/Nikeshbajaj/spkit/master/spkit/data/singing_female_residual.wav" type="audio/wav">
# </audio>
# 
# https://raw.githubusercontent.com/Nikeshbajaj/spkit/master/spkit/data/singing_female_residual.wav